# EVSSM Project


In [1]:
import torch
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

PyTorch version: 2.9.0+cu126
CUDA available: True


In [2]:
!rm -rf /content/EVSSM
!git clone https://github.com/kkkls/EVSSM.git /content/EVSSM
!pip install -q basicsr einops yacs opencv-python tqdm scikit-image timm matplotlib
print('✓ Repo cloned and dependencies installed.')

Cloning into '/content/EVSSM'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (306/306), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 306 (delta 42), reused 249 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (306/306), 682.55 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (42/42), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 23.8 MB/s eta 0:00:00
✓ Repo cloned and dependencies installed.


In [3]:
import sys, os
sys.path.append('/content/EVSSM')
print(os.listdir('/content/EVSSM'))

['test.py', 'VERSION', 'Real_blur_PSNR.py', '.git', 'options', 'environment.yml', 'train.sh', 'README.md', 'license', 'setup.cfg', 'setup.py', 'basicsr', 'test.sh', 'models', 'scripts', '.idea']


In [4]:
!sed -n '1,200p' /content/EVSSM/models/EVSSM.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import numbers
from einops import rearrange, repeat
import math
from mamba_ssm.ops.selective_scan_interface import selective_scan_fn, selective_scan_ref

from torchvision.transforms.functional import resize, to_pil_image  # type: ignore
import numpy as np


def to_3d(x):
    return rearrange(x, 'b c h w -> b (h w) c')


def to_4d(x, h, w):
    return rearrange(x, 'b (h w) c -> b c h w', h=h, w=w)



class WithBias_LayerNorm(nn.Module):
    def __init__(self, normalized_shape):
        super(WithBias_LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        normalized_shape = torch.Size(normalized_shape)

        assert len(normalized_shape) == 1

        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.normalized_shape = normalized_shape

    def for

In [5]:
import sys, os
sys.path.append('/content/EVSSM')

print("Repo contains:", os.listdir('/content/EVSSM'))


Repo contains: ['test.py', 'VERSION', 'Real_blur_PSNR.py', '.git', 'options', 'environment.yml', 'train.sh', 'README.md', 'license', 'setup.cfg', 'setup.py', 'basicsr', 'test.sh', 'models', 'scripts', '.idea']


## **TASK 2 — Simplified EVSSM Model**

In [6]:
%%writefile /content/evssm_nomamba.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange

# -------- LayerNorm --------
class WithBias_LN(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(dim))
        self.bias = nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, keepdim=True, unbiased=False)
        return (x - mean) / (var + 1e-5).sqrt() * self.weight + self.bias

class LN2D(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.ln = WithBias_LN(dim)

    def forward(self, x):
        B,C,H,W = x.shape
        t = rearrange(x, "b c h w -> b (h w) c")
        t = self.ln(t)
        return rearrange(t, "b (h w) c -> b c h w", h=H, w=W)

# -------- EDFFN Block --------
class EDFFN(nn.Module):
    def __init__(self, dim, exp=3):
        super().__init__()
        hid = int(dim * exp)
        self.inp = nn.Conv2d(dim, hid * 2, 1)
        self.dw = nn.Conv2d(hid * 2, hid * 2, 3, padding=1, groups=hid * 2)
        self.out = nn.Conv2d(hid, dim, 1)

    def forward(self, x):
        x = self.inp(x)
        x1, x2 = self.dw(x).chunk(2,1)
        return self.out(F.gelu(x1) * x2)

# -------- TinySS2D Block (Pure PyTorch SSM-like) --------
class TinySS2D(nn.Module):
    def __init__(self, dim, exp=2):
        super().__init__()
        hid = int(dim * exp)
        self.inp = nn.Linear(dim, hid)
        self.dw = nn.Conv2d(hid, hid, 3, padding=1, groups=hid)
        self.ln = nn.LayerNorm(hid)
        self.out = nn.Linear(hid, dim)

    def forward(self, x):
        B,C,H,W = x.shape
        t = rearrange(x, "b c h w -> b h w c")
        t = self.inp(t)
        t2 = rearrange(t, "b h w c -> b c h w")
        t2 = F.gelu(self.dw(t2))
        t2 = rearrange(t2, "b c h w -> b h w c")
        t2 = self.ln(t2)
        t = self.out(t2)
        return rearrange(t, "b h w c -> b c h w")

# -------- EVS Block --------
class EVS(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.ln1 = LN2D(dim)
        self.ssm = TinySS2D(dim)
        self.ln2 = LN2D(dim)
        self.ffn = EDFFN(dim)

    def forward(self, x):
        x = x + self.ssm(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

# -------- Full EVSSM No-Mamba Model --------
class EVSSM_NoMamba(nn.Module):
    def __init__(self, dim=48):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv2d(3, dim, 3, padding=1),
            EVS(dim),
            EVS(dim)
        )
        self.dec = nn.Sequential(
            EVS(dim),
            EVS(dim),
            nn.Conv2d(dim, 3, 3, padding=1)
        )

    def forward(self, x):
        y = self.enc(x)
        y = self.dec(y)
        return y + x  # residual connection


Writing /content/evssm_nomamba.py


In [7]:
from evssm_nomamba import EVSSM_NoMamba
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EVSSM_NoMamba().to(device)

x = torch.randn(1,3,256,256).to(device)
y = model(x)

print("Output:", y.shape)


Output: torch.Size([1, 3, 256, 256])


## **Run Infrence**

In [8]:
%%writefile /content/run_inference.py
import os, torch
from PIL import Image
from torchvision import transforms
from evssm_nomamba import EVSSM_NoMamba

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EVSSM_NoMamba().to(device)
model.eval()

to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

MAX_SIZE = 1024  # safe for Colab GPU

def resize_if_needed(img):
    w, h = img.size
    if max(w, h) > MAX_SIZE:
        scale = MAX_SIZE / max(w, h)
        new_w = int(w * scale)
        new_h = int(h * scale)
        return img.resize((new_w, new_h), Image.LANCZOS)
    return img

input_dir = "/content/blur_20"
output_dir = "/content/outputs_20"
os.makedirs(output_dir, exist_ok=True)

image_files = sorted(os.listdir(input_dir))
print(f"Found {len(image_files)} files\n")

for name in image_files:

    if not name.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff")):
        continue

    print("Processing:", name)
    path = os.path.join(input_dir, name)

    try:
        img = Image.open(path).convert("RGB")
        img = resize_if_needed(img)
    except Exception as e:
        print("❌ ERROR loading", name, "→", e)
        continue

    try:
        x = to_tensor(img).unsqueeze(0).to(device)
        with torch.no_grad():
            pred = torch.clamp(model(x), 0, 1)

        out_img = to_pil(pred.squeeze(0).cpu())
        out_name = name.replace(".png","_out.png")
        out_path = os.path.join(output_dir, out_name)
        out_img.save(out_path)

        print("Saved:", out_path, "\n")

    except Exception as e:
        print("❌ ERROR processing", name, "→", e)
        continue

print("✓ Finished all images!")


Writing /content/run_inference.py


In [9]:
!python /content/run_inference.py

Traceback (most recent call last):
  File "/content/run_inference.py", line 28, in <module>
    image_files = sorted(os.listdir(input_dir))
                         ^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/blur_20'


In [10]:
!pip install -U pip wheel setuptools
!pip install ninja einops packaging

# Install the correct Mamba SSM wheel (works in Colab)
!pip install https://github.com/state-spaces/mamba/releases/download/v1.2.0/mamba_ssm-1.2.0-py3-none-manylinux2014_x86_64.whl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  ERROR: HTTP error 404 while getting https://github.com/state-spaces/mamba/releases/download/v1.2.0/mamba_ssm-1.2.0-py3-none-manylinux2014_x86_64.whl
ERROR: Could not install requirement mamba-ssm==1.2.0 from https://github.com/state-spaces/mamba/releases/download/v1.2.0/mamba_ssm-1.2.0-py3-none-manylinux2014_x86_64.whl because of HTTP error 404 Client Error: Not Found for url: https://github.com/state-spaces/mamba/releases/download/v1.2.0/mamba_ssm-1.2.0-py3-none-manylinux2014_x86_64.whl for URL https://github.com/state-spaces/mamba/releases/download/v1.2.0/mamba_ssm-1.2.0-py3-none-manylinux2014_x86_64.whl


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import torch
import sys

# Install mamba-ssm if not already installed (previous attempt failed)
!pip install mamba-ssm

# Add EVSSM repo to path
sys.path.append('/content/EVSSM')

from models.EVSSM import EVSSM

ckpt_path = "/content/net_g_GoPro.pth"

# Load checkpoint
ckpt = torch.load(ckpt_path, map_location="cpu")

# Some checkpoints store params under ckpt["params"], some under ckpt["state_dict"]
if "params" in ckpt:
    weights = ckpt["params"]
elif "state_dict" in ckpt:
    weights = ckpt["state_dict"]
else:
    weights = ckpt  # fallback

# Initialize model
model = EVSSM()

# Load the weights
model.load_state_dict(weights, strict=True)

# Move to GPU
model = model.cuda().eval()

print("✓ Loaded EVSSM model with net_g_GoPro.pth weights")

ModuleNotFoundError: No module named 'mamba_ssm'